This notebook is to manually check the computation of particular parts of the optimization.

In [6]:
import numpy as np
import pickle
from scipy import stats

# Objective function of $w$ optimization

The objective function has the form
\begin{equation}
    -w^T y + \psi_\alpha \lVert w \rVert_2 - \lambda_k^T K^T w + \frac{\mu}{2} \lVert K^T w \rVert_2^2 - \mu (h + A^T c_k)^T K^T w,
\end{equation}
which to start is equivalent to
\begin{equation}
    -w^T y + \psi_\alpha \lVert w \rVert_2 + \frac{\mu}{2} \lVert K^T w \rVert_2^2 - \mu h^T K^T w,
\end{equation}
since $\lambda_k = c_k = 0$.

As such, I need the following items
1. $w$ --> generated by Gaussian with known random seed
2. $y$ --> affine corrected observation
3. $\psi_\alpha$ --> on Cheyenne
4. $\mu$ --> fixed up front
5. $K^T w$ --> saved in the hash table
6. $h$ --> functional

In [3]:
# affine corrected observation
with open('../data/y_affine_corrected.npy', 'rb') as f:
    y = np.load(f)
m = y.shape[0]
print(y[:10])

[-0.43160245  1.55670918 -0.19545318  1.94694523  1.20822325 -0.39844148
 -1.67594377  0.46664634  0.93883433  2.17580318]


In [10]:
# Gaussian vector
np.random.seed(12345)
w_sp = stats.norm().rvs(m)
print(w_sp[:10])

# make hash value for the above
w_key = hash(w_sp.tobytes())
print(w_key)

[-0.20470766  0.47894334 -0.51943872 -0.5557303   1.96578057  1.39340583
  0.09290788  0.28174615  0.76902257  1.24643474]
1033087255021983398


In [26]:
hash(bytes(234))

5867119967475160626

In [7]:
# slack factor
with open('../data/opt_res_cont.pkl', 'rb') as f:
    opt_slack = pickle.load(f)
PSI2 = stats.chi2.ppf(q=.95, df=1) + opt_slack[1]
psi_alpha = np.sqrt(PSI2)
print(psi_alpha)

145.0614723275884


In [11]:
# table of K^T evals
with open('../data/h_table_lep.pkl', 'rb') as f:
    KTw_dict = pickle.load(f)

In [14]:
# functional
with open('../data/na_june_functional.npy', 'rb') as f:
    h = np.load(f)

In [13]:
KTw_dict.keys()

dict_keys([0, -5402254896407009260, -4245409759836842543, 469256915948961635, -3988124180417985166, 8061287324379625028, 6998018687221583203])

In [18]:
np.linalg.norm(w_sp)

168.56192011973184

In [24]:
KEY = 6998018687221583203
MU = 1e3

t1 = -np.dot(w_sp, y)
t2 = psi_alpha * np.linalg.norm(w_sp)
t3 = (MU / 2) * np.linalg.norm(KTw_dict[KEY])
t4 = MU * np.dot(h, KTw_dict[KEY])

print(f't1 = {t1}')
print(f't2 = {t2}')
print(f't3 = {t3}')
print(f't4 = {t4}')
print(f'Sum = {t1 + t2 + t3 + t4}')

t1 = -471.06566029136116
t2 = 24451.840310933647
t3 = 2.5675250217318535
t4 = -0.08332110450979814
Sum = 23983.258854559506
